In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

# utilsforecast

> Forecasting utilities

## Install

### PyPI
```sh
pip install utilsforecast
```
### Conda
```sh
conda install -c conda-forge utilsforecast
```

## How to use

### Generate synthetic data

In [ ]:
from utilsforecast.data import generate_series

In [ ]:
series = generate_series(3, with_trend=True, static_as_categorical=False)
series

### Plotting

In [ ]:
#| matplotlib
from utilsforecast.plotting import plot_series

In [ ]:
#| matplotlib
fig = plot_series(series, plot_random=False, max_insample_length=50, engine='matplotlib')
fig.savefig('imgs/index.png', bbox_inches='tight')

![](https://raw.githubusercontent.com/Nixtla/utilsforecast/main/nbs/imgs/index.png)

### Preprocessing

In [ ]:
from utilsforecast.preprocessing import fill_gaps

In [ ]:
serie = series[series['unique_id'].eq(0)].tail(10)
# drop some points
with_gaps = serie.sample(frac=0.5, random_state=0).sort_values('ds')
with_gaps

In [ ]:
fill_gaps(with_gaps, freq='D')

### Evaluating

In [ ]:
from functools import partial

import numpy as np

from utilsforecast.evaluation import evaluate
from utilsforecast.losses import mape, mase

In [ ]:
valid = series.groupby('unique_id').tail(7).copy()
train = series.drop(valid.index)
rng = np.random.RandomState(0)
valid['seas_naive'] = train.groupby('unique_id')['y'].tail(7).values
valid['rand_model'] = valid['y'] * rng.rand(valid['y'].shape[0])
daily_mase = partial(mase, seasonality=7)
evaluate(valid, metrics=[mape, daily_mase], train_df=train)